In [222]:
import pandas as pd
import numpy as np
import string

from googletrans import Translator

In [223]:
import nltk
from nltk.corpus import stopwords
 
nltk.download('stopwords')
print(type(stopwords.words('russian')))

<class 'list'>


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Бахтияр\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [224]:
translator = Translator()

punctuations=['?',':','!',',','.',';','|','(',')','--', '-', '#', '"', '№', '%', '«', '»', '–', "'"]
for i in string.punctuation:
    if i not in punctuations:
        punctuations.append(i)

In [225]:
raw_data_articles = pd.read_csv("articles.csv")
raw_data_lif_sit = pd.read_csv("life_situations.csv")
raw_data_news = pd.read_csv("news.csv")
raw_data_services = pd.read_csv("services.csv")

In [226]:
def translate_words(a):
    words = []
    for i in a:
        translated = translator.translate(i, dest='en')
        words.append(translated.text)
    return words

In [227]:
print("ASDFGHJKL"[0:-3])

ASDFGH


In [228]:
def normalize_words(a):
        ans = []
        for i in [x.lower() for x in a.split()]:
            s = ""

            for g in i:
                if g not in punctuations:
                    s += g
                elif g == '-':
                    if len(s) != 0 and not (s in stopwords.words('russian') or s in stopwords.words('english')):
                        if len(s) >= 6:
                            ans.append(s[:-3])
                        else:
                            ans.append(s)
                        s = ""

            if not (s in stopwords.words('russian') or s in stopwords.words('english')):
                if len(s) >= 6:
                    ans.append(s[:-3])
                else:
                    ans.append(s)
        return ans

In [229]:
def standards_words(arr, var, idx):
    words = []
    count = {}
    ans = []
    
    for name in var:
        if len(arr[arr['Unnamed: 0'] == idx][name]) != 0:
            if arr[arr['Unnamed: 0'] == idx][name].isnull().sum() == 0:
                words += normalize_words(arr[arr['Unnamed: 0'] == idx][name].to_list()[0])
                
    for i in words:
        count[i] = count.get(i, 0) + 1
    cnt = 0
    for i in {k: v for k, v in sorted(count.items(), key=lambda item: -item[1])}:
        cnt += 1
        
        if cnt == 500:
            break
        ans.append(i)
    return ans
    

In [230]:
def standards_words_question(question):
    words = []
    count = {}
    ans = []

    words = normalize_words(question)
    
    for i in words:
        count[i] = count.get(i, 0) + 1
    cnt = 0
    for i in {k: v for k, v in sorted(count.items(), key=lambda item: -item[1])}:
        cnt += 1
        ans.append(i)
        if cnt == 500:
            break
    return ans
    

In [ ]:
words = []

for i in raw_data_articles['Unnamed: 0'].to_list():
    test = standards_words(raw_data_articles, ['title', 'content'], i)
    words.append(test)

In [ ]:
raw_data_articles['Words'] = words

In [ ]:
raw_data_articles.drop(['id', 'projects', 'title', 'content'], axis=1, inplace=True)

In [ ]:
raw_data_articles

In [ ]:
raw_data_news

In [ ]:
words = []

for i in raw_data_news['Unnamed: 0'].to_list():
    test = standards_words(raw_data_news, ['title', 'body'], i)
    words.append(test)

In [ ]:
raw_data_news['Words'] = words
raw_data_news.drop(['id', 'projects', 'title', 'body', 'short_description'], axis=1, inplace=True)

In [ ]:
words = []

for i in raw_data_services['Unnamed: 0'].to_list():
    test = standards_words(raw_data_services, ['additional_info', 'description', 'full_title', 'short_title', 'title', 'result_description'], i)
    words.append(test)

In [ ]:
raw_data_services['Words'] = words
raw_data_services.drop(['id', 'additional_info', 'description', 'full_title', 'short_title', 'title', 'result_description'], axis=1, inplace=True)

In [ ]:
words = []

for i in raw_data_lif_sit['Unnamed: 0'].to_list():
    test = standards_words(raw_data_lif_sit, ['intro', 'title_main', 'title_sub', 'instruction'], i)
    words.append(test)

In [ ]:
raw_data_lif_sit['Words'] = words
raw_data_lif_sit.drop(['id', 'intro', 'title_main', 'title_sub', 'subid', 'instruction'], axis=1, inplace=True)

In [ ]:
raw_data_lif_sit.rename(columns={"URL": "url"}, inplace=True)

In [ ]:
data = pd.concat([raw_data_articles, raw_data_lif_sit, raw_data_news, raw_data_services])
data

In [ ]:
raw_data_train = pd.read_csv("epir_test.csv")

In [ ]:
data_kk =  data[data['sys_lang'] == 'kk']

In [ ]:
data_ru = data[data['sys_lang'] == 'ru']

In [ ]:
data_qq = data[data['sys_lang'] == 'qq']

In [ ]:
data_en = data[data['sys_lang'] == 'en']

In [ ]:
answers_idx = []

for question in raw_data_train["question"]: 
    idx = 0
    index = 0
    mx = 0

    array_ques_words = standards_words_question(question)

    eng = True

    for i in array_ques_words[0]:
        if not(i >= 'a' and i <= 'z'):
            eng = False
            break
    if eng:
        for answers in data_en:
            cnt = 0

            for q_word in array_ques_words:
                if q_word in answers:
                    cnt += 1
            if cnt > mx:
                mx = cnt
                index = idx
            idx += 1
        answers_idx.append(index)
    else:
        kaz = False

        for q_word in array_ques_words:
                for let in q_word:
                    if let in 'әіңғүұқөһ':
                        kaz = True
                        break

        if kaz:
            for answers in data_kk:
                cnt = 0

                for q_word in array_ques_words:
                    if q_word in answers:
                        cnt += 1
                if cnt > mx:
                    mx = cnt
                    index = idx
                idx += 1
            answers_idx.append(index)
        else:
            for answers in data_ru:
                cnt = 0

                for q_word in array_ques_words:
                    if q_word in answers:
                        cnt += 1
                if cnt > mx:
                    mx = cnt
                    index = idx
                idx += 1
            answers_idx.append(index)

In [ ]:
answers_idx

last_answers = []

for i in answers_idx:
    last_answers.append(data[i:i+1]["Unnamed: 0"].to_list())

last_answers

In [ ]:
answers_idx = []
for i in last_answers:
    answers_idx.append(i[0])

In [ ]:
answers_idx

In [202]:
answers_idx

[147888,
 38519,
 8753,
 12759,
 10394,
 7810,
 7810,
 18349,
 10551,
 10551,
 54427,
 4702,
 836,
 836,
 83335,
 49673,
 52061,
 6804,
 6107,
 65206,
 88347,
 77922,
 22302,
 6770,
 67298,
 38032,
 45863,
 45863,
 3469,
 89287,
 58364,
 22302,
 19961,
 9789,
 52967,
 3948,
 29658,
 50409,
 28136,
 4698,
 7810,
 199653,
 10600,
 5992,
 18095,
 16919,
 7126,
 3408,
 23101,
 65497,
 55325,
 50734,
 11653,
 98539,
 95413,
 174426,
 30632,
 184512,
 25844,
 184512,
 129263,
 4424,
 106870,
 57300,
 6485,
 14250,
 10381,
 3077,
 39381,
 87269,
 67501,
 7533,
 16468,
 4206,
 3099,
 2885,
 2885,
 207030,
 118251,
 19384,
 3099,
 27546,
 38010,
 3469,
 204861,
 206136,
 29896,
 4055,
 16843,
 52021,
 135900,
 3543,
 11280,
 13016,
 11063,
 4076,
 4717,
 5072,
 2942,
 11447,
 9069,
 5067,
 5172,
 205024,
 22978,
 206037,
 10205,
 203676,
 4702,
 203676,
 38519,
 38519,
 4702,
 203225,
 38519,
 204204,
 6787,
 34420,
 204204,
 2987,
 205598,
 128617,
 3405,
 9926]

In [203]:
raw_data_train['index'] = answers_idx

In [204]:
raw_data_train.drop(['question'], axis=1, inplace=True)

In [205]:
raw_data_train.to_csv('res3.csv', index=False)